In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('train_E6oV3lV.csv')

In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
train.isnull().count()

id       31962
label    31962
tweet    31962
dtype: int64

In [5]:
test = pd.read_csv('test_tweets_anuFYb8.csv')

In [6]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [7]:
import nltk

In [8]:
train_df = train['tweet']
train_df.head()

0     @user when a father is dysfunctional and is s...
1    @user @user thanks for #lyft credit i can't us...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

In [9]:
from nltk import sent_tokenize

In [10]:
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [11]:
stemmer =PorterStemmer()

In [12]:
import string

mess = 'Sample message! Notice: it has punctuation.'

# Check characters to see if they are in punctuation
nopunc = [char for char in mess if char not in string.punctuation]

# Join the characters again to form the string.
nopunc = ''.join(nopunc)

In [13]:
from nltk.corpus import stopwords

In [14]:
nopunc.split()

['Sample', 'message', 'Notice', 'it', 'has', 'punctuation']

In [15]:
clean_mess = [stemmer.stem(word) for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [16]:
clean_mess

['sampl', 'messag', 'notic', 'punctuat']

In [17]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [18]:
tweet = train['tweet'].apply(text_process)

In [19]:
tweet.head()

0    [user, father, dysfunctional, selfish, drags, ...
1    [user, user, thanks, lyft, credit, cant, use, ...
2                                    [bihday, majesty]
3    [model, love, u, take, u, time, urð±, ðð...
4                    [factsguide, society, motivation]
Name: tweet, dtype: object

In [25]:
from sklearn.model_selection import train_test_split

twt_train, twt_test, label_train, label_test = \
train_test_split(train['tweet'], train['label'], test_size=0.2)

print(len(twt_train), len(twt_test), len(twt_train) + len(twt_test))

25569 6393 31962


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
bow_vect = TfidfVectorizer(analyzer=text_process)

# Print total number of vocab words

In [28]:
bow = bow_vect.fit_transform(twt_train)

In [30]:
from sklearn.naive_bayes import MultinomialNB
sentiment_model = MultinomialNB().fit(bow, label_train)

In [36]:
all_predictions = sentiment_model.predict(bow)
print(all_predictions)

[0 0 0 ... 0 0 0]


In [39]:
from sklearn.metrics import classification_report
print (classification_report(label_train, all_predictions))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     23763
           1       1.00      0.19      0.33      1806

    accuracy                           0.94     25569
   macro avg       0.97      0.60      0.65     25569
weighted avg       0.95      0.94      0.92     25569



In [40]:
from sklearn.svm import LinearSVC

In [42]:
clf= LinearSVC()

In [57]:
model = clf.fit(bow,label_train)

In [58]:
predictions = model.predict(bow)

In [59]:
print (classification_report(label_train, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23763
           1       1.00      0.97      0.98      1806

    accuracy                           1.00     25569
   macro avg       1.00      0.98      0.99     25569
weighted avg       1.00      1.00      1.00     25569



In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
rfc = RandomForestClassifier()

In [60]:
model2 =rfc.fit(bow,label_train)

In [76]:
prediction = model2.predict(bow)

In [62]:
print (classification_report(label_train, prediction))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     23763
           1       1.00      1.00      1.00      1806

    accuracy                           1.00     25569
   macro avg       1.00      1.00      1.00     25569
weighted avg       1.00      1.00      1.00     25569



In [81]:
from sklearn.model_selection import train_test_split

twt_train, twt_test, label_train, label_test = \
train_test_split(train['tweet'], train['label'], test_size=0.2)

print(len(twt_train), len(twt_test), len(twt_train) + len(twt_test))

25569 6393 31962


In [70]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('bow', TfidfVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [71]:
pipeline.fit(twt_train,label_train)

Pipeline(memory=None,
         steps=[('bow',
                 TfidfVectorizer(analyzer=<function text_process at 0x00000205E24F6D38>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0

In [72]:
predictions = pipeline.predict(test['tweet'])

In [73]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [74]:
pred=pd.DataFrame(predictions)
sub_df=pd.read_csv('test_tweets_anuFYb8.csv')
datasets=pd.concat([sub_df['id'],pred],axis=1)
datasets.columns=['id','label']
datasets.to_csv('sample_submission.csv',index=False)